In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from instanseg import InstanSeg
from instanseg.utils.utils import show_images
from napari_spatialdata import Interactive
from spatialdata import SpatialData
import numpy as np
import torch
import napari

torch.cuda.is_available()

True

In [3]:
instanseg_fluorescence = InstanSeg("fluorescence_nuclei_and_cells", verbosity=1)

Model fluorescence_nuclei_and_cells version 0.1.0 already downloaded in c:\Users\KMK280\AppData\Local\miniconda3\envs\instaseg-env\Lib\site-packages\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda


In [ ]:
FILE_PATH = r"D:\multiplex_pipeline\notebooks\Core_018.zarr"
sdata = SpatialData.read(FILE_PATH)

In [5]:
selected_signals = ['001_DAPI','HLA1','CD8a','CD20','NaKATPase','TGFb1','wee1','pCK26','pS6']

In [6]:
channel_arrays = []
for channel in selected_signals:
    if channel in sdata.images:
        img = np.array(sdata[channel].scale0.image).squeeze()
        # rescale the image beteen 1 and 99 percentiles
        p1, p99 = np.percentile(img, (1, 99.5))
        img = np.clip(img, p1, p99)
        img = (((img - p1) / (p99 - p1))*255).astype(int)
        channel_arrays.append(img)
    else:
        print(f"Warning: Channel {channel} not found in SpatialData object.")

# Stack into a single 3D numpy array with shape (z, y, x)
# where z is the number of selected channels
stacked_array = np.stack(channel_arrays, axis=2)

In [7]:
stacked_array.shape

(5888, 6272, 9)

In [ ]:
# run test on a patch of the image

torch.cuda.empty_cache()

labeled_output, image_tensor  = instanseg_fluorescence.eval_small_image(stacked_array[2000:3500,2000:3500,:], pixel_size = 0.3, 
                                                                        resolve_cell_and_nucleus=True, 
                                                                        cleanup_fragments = True)

In [11]:
labels_1 = np.array(labeled_output[0,0,:,:]).astype(int)
labels_2 = np.array(labeled_output[0,1,:,:]).astype(int)

viewer = napari.Viewer()
viewer.add_image(np.moveaxis(stacked_array,-1,0),colormap='viridis',blending='additive')
viewer.add_labels(labels_1, translate=[2000,2000])
viewer.add_labels(labels_2, translate=[2000,2000])

<Labels layer 'labels_2' at 0x2ab6eaacd90>